In [7]:
import numpy as np
import random

def eig(A_original, max_iter = 10):
    A = A_original.copy().astype(float)
    current_A = A.copy()
    n = A.shape[0]
    eigenvalues = []
    eigenvectors = np.zeros_like(A)
    for i in range(n):
        v = np.random.rand(n)
        mu = np.random.normal(0, 10)
        if i == 1:
            current_A = A-eigenvalues[i-1] * np.outer(eigenvectors[:,i-1], eigenvectors[:,i-1].T)
        if i > 1:
            current_A = current_A - np.outer(eigenvectors[:,i-1], eigenvectors[:,i-1].T)
        #print('i=', i, 'A\n', current_A)
        iterations = 0
        while max(abs(current_A@v-mu*v))>2**(-10) and\
            iterations < max_iter:
            v = np.linalg.solve(current_A-mu*np.identity(n), v)
            v = v/np.linalg.norm(v)
            mu = v.T@current_A@v
            if abs(mu)<2**(-10):
                mu = np.random.randn()
            iterations += 1
        if iterations >= max_iter:
            print('nie znaleziono')
        eigenvectors[:,i] = v
        eigenvalues.append(mu)
    return (eigenvalues, eigenvectors)

In [12]:
A = np.array([[1, 2, 3],
              [2, 5, 4],
              [3, 4, 10]])
x = np.linalg.eig(A)
print(x)
print(np.linalg.eig(A-x[0][0]*np.outer(x[1][:, 0], x[1][:, 0].T)))

(array([13.27009751, -0.08044285,  2.81034534]), array([[ 0.28122477,  0.95520076, -0.09221791],
       [ 0.47210705, -0.22137446, -0.85329261],
       [ 0.83548044, -0.1964303 ,  0.51321299]]))
(array([ 2.81034534e+00, -8.04428504e-02, -2.51787677e-15]), array([[ 0.09221791,  0.95520076,  0.28122477],
       [ 0.85329261, -0.22137446,  0.47210705],
       [-0.51321299, -0.1964303 ,  0.83548044]]))


In [11]:
A = np.array([[1, 2, 3],
              [2, 5, 4],
              [3, 4, 10]])
nasze_values, nasze_wektors = eig(A, max_iter = 100)

print('Kod:\n', nasze_values, '\n', nasze_wektors, '\nlinalg:\n', np.linalg.eig(A))

nie znaleziono
Kod:
 [2.810345315892716, -0.3899555987176914, 13.270097514127814] 
 [[ 0.09213888 -0.09397679  0.28122459]
 [ 0.85331599 -0.85288457  0.47210534]
 [-0.51318831  0.51357207  0.83548146]] 
linalg:
 (array([13.27009751, -0.08044285,  2.81034534]), array([[ 0.28122477,  0.95520076, -0.09221791],
       [ 0.47210705, -0.22137446, -0.85329261],
       [ 0.83548044, -0.1964303 ,  0.51321299]]))


In [4]:
#A = np.array([[1, 0],
#              [0, 1]])
A = np.array([[1, 2, 3],
              [2, 5, 4],
              [3, 4, 10]])
nasze_values, nasze_wektors = eig(A)

print('Kod:\n', nasze_values, '\n', nasze_wektors, '\nlinalg:\n', np.linalg.eig(A))

nie znaleziono
Kod:
 [13.270097514241566, -0.04721582859449729, -1.000000212547373] 
 [[ 0.28122458 -0.28269953  0.28282965]
 [ 0.47210462 -0.47175295  0.47172268]
 [ 0.83548187 -0.83518269  0.83515574]] 
linalg:
 (array([13.27009751, -0.08044285,  2.81034534]), array([[ 0.28122477,  0.95520076, -0.09221791],
       [ 0.47210705, -0.22137446, -0.85329261],
       [ 0.83548044, -0.1964303 ,  0.51321299]]))


In [5]:
def random_matrices(columns, rows, quantity=1):
    result = []
    for i in range(quantity):
        result.append(np.random.rand(rows, columns))
    return np.array(result)

In [6]:
n = 1000
dim_max = 10
i = 0
for dim in range(1, dim_max+1):
    for B in random_matrices(dim, dim, n):
        array = np.matmul(A, A.T)
        result = eig(array)
        if np.all(np.isclose(result.T@result, np.identity(dim)) == True):
            i+=1
        else:
            print('array', array, '\nwynik funkcji\n', result.T@result)
print('Otrzymano', i, 'poprawnych wyników na', n*dim_max, 'testów.')

nie znaleziono


AttributeError: 'tuple' object has no attribute 'T'

In [ ]:
def svd(A):
    W = A
    Q = A
    V = A
    return (W, Q, V)